In [3]:
import equinox as eqx
import jax
import jax.numpy as jnp
from dataclasses import dataclass

In [4]:
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

Platform 'METAL' is experimental and not all JAX functionality may be correctly supported!


Metal device set to: Apple M3 Pro

systemMemory: 18.00 GB
maxCacheSize: 6.00 GB

METAL


W0000 00:00:1718494140.626312   28442 mps_client.cc:510] WARNING: JAX Apple GPU support is experimental and not all JAX functionality is correctly supported!
I0000 00:00:1718494140.638435   28442 service.cc:145] XLA service 0x11b22ab20 initialized for platform METAL (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1718494140.638448   28442 service.cc:153]   StreamExecutor device (0): Metal, <undefined>
I0000 00:00:1718494140.641471   28442 mps_client.cc:406] Using Simple allocator.
I0000 00:00:1718494140.641486   28442 mps_client.cc:384] XLA backend will use up to 12883132416 bytes on device 0 for SimpleAllocator.


In [42]:
key = jax.random.PRNGKey(0)
a = jax.random.normal(key=key, shape=(5, 4, 3))
# b = jnp.zeros((3, 4, 10, 5,))
# jnp.dot(a, b).shape

In [43]:
a

Array([[[-1.152883  , -1.229587  , -0.7161828 ],
        [ 1.6050261 , -0.9966271 ,  1.0175147 ],
        [-2.218356  , -0.3283949 , -0.4063526 ],
        [ 0.31958023,  0.54812   , -0.982553  ]],

       [[-0.5400347 , -2.8863804 ,  0.12353314],
        [ 1.0260264 ,  1.4461187 ,  0.10816291],
        [-0.25177038,  1.2097973 ,  1.6453063 ],
        [ 0.04605093,  0.353377  , -1.5007296 ]],

       [[ 0.36558455, -0.47383544, -0.79497063],
        [-0.63448733, -0.65156275,  0.5933108 ],
        [-1.5331371 , -0.9754873 , -1.9289044 ],
        [-1.0240635 , -0.6569473 , -1.8061997 ]],

       [[-0.47573346, -0.43003812,  1.630926  ],
        [ 1.4903384 ,  0.63752794, -0.8156754 ],
        [-1.1004527 ,  0.19227722, -1.6322023 ],
        [ 0.9158138 ,  0.08496428,  1.0495372 ]],

       [[ 1.8179129 ,  1.3885509 , -0.42771345],
        [ 1.7168485 ,  1.6853313 ,  1.4082001 ],
        [ 0.149996  , -0.18189907,  1.3802307 ],
        [-0.1458    ,  0.46899822, -0.09020927]]], dtype=floa

In [21]:
if (x := jax.devices()) is not None:
    print(x)

[METAL(id=0)]


In [19]:
a = jnp.zeros((100, 100), dtype=jnp.int4)

XlaRuntimeError: UNKNOWN: <unknown>:0: error: 'func.func' op One or more function input/output data types are not supported.
<unknown>:0: note: see current operation: 
"func.func"() <{arg_attrs = [{mhlo.layout_mode = "default"}], function_type = (tensor<i4>) -> tensor<i4>, res_attrs = [{jax.result_info = "", mhlo.layout_mode = "default"}], sym_name = "main", sym_visibility = "public"}> ({
^bb0(%arg0: tensor<i4>):
  "func.return"(%arg0) : (tensor<i4>) -> ()
}) : () -> ()
<unknown>:0: error: failed to legalize operation 'func.func'
<unknown>:0: note: see current operation: 
"func.func"() <{arg_attrs = [{mhlo.layout_mode = "default"}], function_type = (tensor<i4>) -> tensor<i4>, res_attrs = [{jax.result_info = "", mhlo.layout_mode = "default"}], sym_name = "main", sym_visibility = "public"}> ({
^bb0(%arg0: tensor<i4>):
  "func.return"(%arg0) : (tensor<i4>) -> ()
}) : () -> ()


In [3]:
@dataclass 
class RNNConfig: 
  input_dim: int = 2
  output_dim: int = 1
  hidden_dim: int = 40 

In [4]:
key = jax.random.PRNGKey(446)

class RNN(eqx.Module):
    config: RNNConfig
    hidden_dim: int
    Wh: eqx.nn.Linear
    Wx: eqx.nn.Linear
    Wy: eqx.nn.Linear
    act: callable
    device: str = 'cpu'
    
    # TODO: check bias 
    def __init__(self, config, device='cpu'):
        self.config = config
        self.hidden_dim = config.hidden_dim
        self.Wh = eqx.nn.Linear(config.hidden_dim, config.hidden_dim, key=key)
        self.Wx = eqx.nn.Linear(config.input_dim, config.hidden_dim, key=key)
        self.Wy = eqx.nn.Linear(config.hidden_dim, config.output_dim, key=key)
        self.act = jax.nn.sigmoid
        self.device = device

    def __call__(self, x): 
        # batch_size, seq_len, _ = x.shape
        # batch_size, seq_len, _  = 1, x.shape
        seq_len, _ = x.shape

        # hidden = torch.nn.Parameter(torch.zeros(batch_size, self.hidden_dim))
        # hidden = jnp.zeros((batch_size, self.hidden_dim))
        hidden = jnp.zeros((self.hidden_dim,))
        outs = []

        for i in range(seq_len):
            # print(self.Wh(hidden).shape)
            # print(hidden.shape)
            hidden = self.act(self.Wh(hidden) + self.Wx(x[i,:]))
            out = self.Wy(hidden)
            outs.append(out)
        
        # output shape = (batch_size, sequence length, output dimension)
        # return torch.stack(outs).permute(1,0,2)
        stacked = jnp.stack(outs, axis=1)
        return stacked
        # return jnp.transpose(stacked, (1,0,2))


In [5]:
config = RNNConfig(input_dim=2, hidden_dim=2, output_dim=1)
model = RNN(config)
model = jax.vmap(model)

In [8]:
batch_size = 128
seq_len = 10
input_dim = 2
# x = jax.random.normal(key, (batch_size, seq_len, input_dim))
x = jax.random.normal(key, (seq_len, input_dim))
model(x)

ValueError: not enough values to unpack (expected 2, got 1)

In [4]:
type(type(128))

type